In [1]:
import torch, torchvision
import torchvision.transforms.functional as fn
torch.manual_seed(0)

import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Update os
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

Preprocessing

In [3]:
#Create labels: cats=0, dogs=1
labels = np.zeros(25_000)
labels[12_500:] =  1

In [4]:
#create augmentations for ImageDataset
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

transformations = torch.nn.Sequential(
        torchvision.transforms.Resize((300,300)),
        torchvision.transforms.RandomPerspective(p=.25),
        torchvision.transforms.RandomHorizontalFlip(p=.25)

)

#Build training dataset
training_data = ImageDataset(image_paths= "/home/kishen/documents/python_projects/dogs_v_cats/data/train/",
                            targets=labels,
                            augmentations= transformations)



In [5]:
from torch.utils.data import DataLoader

loader_params = {"batch_size": 16,
                 "shuffle": True,
                 "num_workers": 16}
train_loader = DataLoader(training_data, **loader_params)

**Training**

In [6]:
#set hyperparameters
epochs = 5
criterion = torch.nn.BCEWithLogitsLoss()
model_params={'lr':.0001,
             'momentum':.9}

#init model and optimizer
from src.models import ConvNet

model = ConvNet()
optimizer = torch.optim.SGD(model.parameters(), **model_params)

#move model to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cuda


In [9]:
epoch_loss = []
for epoch in range(epochs):
    running_loss = 0

    for i, data in enumerate(train_loader):
        
        #move data to gpu
        inputs, labels = data['image'], data['target']
        inputs, labels = inputs.to(device), labels.float().to(device)

        #zero gradients
        optimizer.zero_grad() 

        #model computations
        preds = model(inputs.type(torch.cuda.FloatTensor))

        loss = criterion((preds), labels.unsqueeze(1))

        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    epoch_loss.append(running_loss / i)
print("Training complete!")



Epoch 1: 0.0053
Epoch 1: 0.6634
Epoch 1: 0.6045
Epoch 1: 0.597
Epoch 2: 0.0008
Epoch 2: 0.5786
Epoch 2: 0.5622
Epoch 2: 0.5433
Epoch 3: 0.0012
Epoch 3: 0.519
Epoch 3: 0.5116
Epoch 3: 0.514
Epoch 4: 0.0009
Epoch 4: 0.486
Epoch 4: 0.4789
Epoch 4: 0.4676
Epoch 5: 0.0008
Epoch 5: 0.4411
Epoch 5: 0.4436
Epoch 5: 0.4356


In [ ]:
print(labels)

tensor([1., 0., 0., 1., 0., 0., 1., 0.], device='cuda:0')
